# Implementation with the RandomForestClassifier

In [1]:
import os
import numpy as np

import tensorflow as tf
print("TensorFlow version:", tf.__version__)

np.set_printoptions(precision=4)
seg_length = 500

TensorFlow version: 2.15.0


In [2]:
X = None
y = None

db1_pos = ['chfdb']
db1_neg = ['nsrdb', 'fantasia']
db2_pos = ['chf2db']
db2_neg = ['nsr2db']

for db in db1_pos:
    for record in os.listdir(f'data/{db}/rr'):
        rr = np.loadtxt(f'data/{db}/rr/{record}')
        n_segments = rr.shape[0] // seg_length
        rr = rr[:n_segments*seg_length].reshape((n_segments, seg_length))
        labels = np.c_[np.ones(n_segments), np.zeros(n_segments)]

        if X is None or y is None:
            X = rr
            y = labels
        else:
            X = np.r_[rr, X]
            y = np.r_[labels, y]

for db in db1_neg:
    for record in os.listdir(f'data/{db}/rr'):
        rr = np.loadtxt(f'data/{db}/rr/{record}')
        n_segments = rr.shape[0] // seg_length
        rr = rr[:n_segments*seg_length].reshape((n_segments, seg_length))
        labels = np.c_[np.zeros(n_segments), np.ones(n_segments)]

        if X is None or y is None:
            X = rr
            y = labels
        else:
            X = np.r_[rr, X]
            y = np.r_[labels, y]

X.shape, y.shape

((7397, 500), (7397, 2))

In [3]:
# from sklearn.model_selection import train_test_split
# from sklearn import preprocessing

# # normalize?
# # X = (X - X.mean()) / X.std()
# X = preprocessing.scale(X, axis=1)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=410)
# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=410)

# # scaler = preprocessing.StandardScaler()
# # scaler.fit(X_train)
# # X_train = scaler.transform(X_train)
# # X_valid = scaler.transform(X_valid)
# # X_test = scaler.transform(X_test)

# X_train.shape, X_valid.shape, X_test.shape

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=410)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=410)

X_train.shape, X_valid.shape, X_test.shape

((5991, 500), (666, 500), (740, 500))

In [5]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
X_train_scaled = min_max_scaler.fit_transform(X_train)
X_test_scaled = min_max_scaler.transform(X_test)

print(min_max_scaler.scale_)

[0.6312 0.3414 0.6281 0.6813 0.2707 0.5169 0.675  0.6591 0.3743 0.6754
 0.6407 0.6641 0.6345 0.3811 0.646  0.6465 0.6419 0.6766 0.6733 0.6281
 0.6626 0.1971 0.612  0.676  0.473  0.6535 0.6454 0.4377 0.6424 0.6622
 0.6813 0.6931 0.3879 0.6692 0.6249 0.5907 0.2291 0.6735 0.3371 0.3052
 0.3935 0.6216 0.3751 0.614  0.5895 0.581  0.6023 0.3591 0.6455 0.6186
 0.3051 0.5765 0.5924 0.5224 0.6325 0.4695 0.5826 0.62   0.663  0.4731
 0.0022 0.2824 0.6914 0.489  0.2744 0.6969 0.6263 0.6751 0.3834 0.3782
 0.6393 0.6405 0.6569 0.4778 0.6482 0.6811 0.7222 0.719  0.3842 0.6667
 0.6719 0.4183 0.0022 0.4842 0.6546 0.5747 0.6153 0.6922 0.4841 0.5424
 0.3619 0.651  0.3736 0.6908 0.6095 0.5036 0.3753 0.585  0.1945 0.4848
 0.598  0.4708 0.6062 0.2654 0.2931 0.4743 0.3007 0.2582 0.599  0.3822
 0.5923 0.6431 0.5537 0.4881 0.6791 0.6679 0.5965 0.477  0.6277 0.5132
 0.6185 0.5095 0.6418 0.625  0.6202 0.6153 0.4911 0.5894 0.5811 0.6183
 0.6186 0.6576 0.6294 0.4003 0.247  0.6081 0.6482 0.5055 0.3686 0.5965
 0.646

In [6]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=101, n_estimators=150)
model.fit(X_train_scaled, y_train)

# training accuracy
print('Accuracy for the train data', model.score(X_train_scaled, y_train))

Accuracy for the train data 1.0


In [7]:
from sklearn import metrics
y_pred = model.predict(X_test_scaled)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# print("*** Confusion Matrix ***")
# print(metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.8932432432432432


In [8]:
# X_train.max(), X_train.min()

In [9]:
# X_train.mean(), X_train.std()

In [10]:
# inputs = tf.keras.layers.Input(shape=(500, 1))


# def inception_lstm(inputs):
#     a = tf.keras.layers.Bidirectional(
#         tf.keras.layers.LSTM(5, return_sequences=True))(inputs)
#     a = tf.keras.layers.LSTM(5, return_sequences=True)(a)

#     b = tf.keras.layers.Conv1D(5, kernel_size=1, padding='same')(inputs)
#     b = tf.keras.layers.Conv1D(5, kernel_size=3, padding='same')(b)

#     c = tf.keras.layers.Conv1D(5, kernel_size=1, padding='same')(inputs)
#     c = tf.keras.layers.Conv1D(5, kernel_size=5, padding='same')(c)

#     d = tf.keras.layers.MaxPool1D(3)(inputs)
#     d = tf.keras.layers.Conv1D(5, kernel_size=1)(d)

#     concat = tf.keras.layers.Concatenate(1)([a, b, c, d])
#     return concat


# layer1 = inception_lstm(inputs)
# layer2 = inception_lstm(layer1)
# dropout = tf.keras.layers.Dropout(0.2)(layer2)
# flattened = tf.keras.layers.Flatten()(dropout)
# output = tf.keras.layers.Dense(2, activation='sigmoid')(flattened)

# model = tf.keras.Model(inputs, output)
# model.summary()

In [11]:
# loss_fn = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer='adam',
#               loss=loss_fn,
#               metrics=['accuracy'])

In [12]:
# np.unique(y_test.argmax(axis=1), return_counts=True)

In [13]:
# y_pred = model.predict(X_test)

In [14]:
# np.unique(y_pred.argmax(axis=1), return_counts=True)

In [15]:
# # callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# model.fit(x=X_train, y=y_train, validation_data=(X_valid, y_valid),
#           batch_size=128, epochs=100)

In [16]:
# model.save('model4.keras')

# model 3, minmax sample scaling w/ no early stopping (100 epochs)
# model 4, zero mean feature scaling w/ no early stopping (100 epochs)

In [17]:
# from sklearn.metrics import recall_score
# from sklearn.metrics import precision_score
# from sklearn.metrics import accuracy_score

# y_pred = model.predict(X_test).argmax(axis=1)

In [18]:
# print(recall_score(y_test.argmax(axis=1), y_pred))
# print(precision_score(y_test.argmax(axis=1), y_pred))
# print(accuracy_score(y_test.argmax(axis=1), y_pred))